# Continuous Training
Now that the data, the network frameworks and the training loops are set up, we can investigate fruther.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

import sys; sys.path.insert(0, '..')
from lib import data, networks, training, utils

## Load preprocessed dataset

In [2]:
train_ds_pre, train_ds_post, test_ds = data.load_data()

## Initialize the models

In [3]:
num_classes = 10
# Small model
model1 = networks.NN([128, 128], num_classes)
# Broad Model
model2 = networks.NN([512], num_classes)
# Mixed Model
model3 = networks.NN([256, 256], num_classes)
# cnn
model4 = networks.CNN([(32, 3), (64, 5), (128, 7)], num_classes)
# cnn small
model5 = networks.CNN([(32, 3), (64, 5)], num_classes)
# ensemble
ensemble = networks.Ensemble([model1, model2, model3, model4, model5])

## Load the weights from pre-training

In [4]:
model1.load_weights('../models/NN128128')
model2.load_weights('../models/NN512')
model3.load_weights('../models/NN256256')
model4.load_weights('../models/CNN3264128')
model5.load_weights('../models/CNN3264')

## Training more

In [ ]:
_,_,_ = training.pretraining(model1, train_ds_pre, test_ds, 10)

In [ ]:
_,_,_ = training.pretraining(model2, train_ds_pre, test_ds, 10)

In [ ]:
_,_,_ = training.pretraining(model3, train_ds_pre, test_ds, 10)

In [ ]:
_,_,_ = training.pretraining(model4, train_ds_pre, test_ds, 2)

In [ ]:
_,_,_ = training.pretraining(model5, train_ds_pre, test_ds, 4)

## Ensemble accuracy after pre-training

In [11]:
_, acc = training.test(ensemble, test_ds, tf.keras.losses.CategoricalCrossentropy())
acc

0.9650909810126582

## Run Post-training dataset through ensemble

In [12]:
utils.run_data(ensemble, train_ds_post)

## Check how much data we gathered to train on

In [14]:
len(ensemble.get_data())

1306

## Check how much data we could not predict

In [15]:
len(ensemble.get_missed_data())

219

## Training individual networks on the collected data

In [17]:
_,_,_ = training.posttraining(ensemble, test_ds, epochs=1)

Model: __ 0
Epoch: __ 0
LOSS 0.09365696698586351 ::: ACC 0.9708267405063291
Model: __ 1
Epoch: __ 0
LOSS 0.09418028501109986 ::: ACC 0.9721123417721519
Model: __ 2
Epoch: __ 0
LOSS 0.08310858464852872 ::: ACC 0.9750791139240507
Model: __ 3
Epoch: __ 0
LOSS 0.038494817867633424 ::: ACC 0.9880340189873418
Model: __ 4
Epoch: __ 0
LOSS 0.20575335277598114 ::: ACC 0.9347310126582279


## Testing ensemble after post-training

In [18]:
_, acc = training.test(ensemble, test_ds, tf.keras.losses.CategoricalCrossentropy())
acc

0.9691455696202531

## TODO
- ensemble acc
- Collect performance for 20,40,60,80,99 acc
- wider varying accuracies